<a href="https://colab.research.google.com/github/ConnectNitish/ML_hack_Qipl/blob/main/Nitish_Neural_Network_Intermedate_Crop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ID - Unique number represents each record - Unique

Insects - Average number of insects that (may) affect the crop. - Numerical

Crop - Type of crop [Food crop and Feed crop] - Categorical

Soil - Type of soil [Clay and silt] - Categorical

Category_of_Toxicant - Values 1, 2, 3 represent three different toxicants available in the market. - Categorical

Does_count - Represents the dosage count of toxicants. - Numerical

Number_of_Weeks_Used - Represents the number of weeks for which the crop remained toxicant. - Numerical

Number_Weeks_does_not_used - Represents the number of weeks that the toxicants are not used on the crop(No. of weeks crop remained without applying any toxicants.) - Numerical

Season - Represents the various seasons (Monsoon, etc.,) - categorical

Label :

Crop_status - 0 represents ‘Crop can be used further’, 1 represents ‘crop is damaged due to nature’, 2 represents ‘toxicants are responsible for crop damage’. - Categorical - Logisitc regression

TO DO : Remove NA Column and Then Train



**Import Package**

In [50]:
import pandas as pd
# seaborn is used for visulazation 
import seaborn as sns  
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf


from sklearn import metrics

import numpy as np

# allow plots to appear directly in the notebook
%matplotlib inline



In [51]:
ls

sample_data/


In [52]:
url = 'https://raw.githubusercontent.com/matadeenmishra/ML_hack_Qipl/main/Sample_Submission_crop_without_season.csv'
# url = "over_2.csv"
url = 'https://raw.githubusercontent.com/ConnectNitish/ML_hack_Qipl/main/Intermediate/OVER_Sample.csv'
data = pd.read_csv(url)
# data.head()

## All the Columns Present In data Set

In [53]:
data.columns

Index(['Insects', 'Crop', 'Soil', 'Category_of_Toxicant', 'Does_count',
       'Number_of_Weeks_Used', 'Number_Weeks_does_not used', 'Crop_status'],
      dtype='object')

In [54]:
data.shape

(200148, 8)

## Missing Values : Give the Values of All NA Columns in Sorted Order

Give all the columns who has NA Values greater than 1% 

In [55]:
data.isna().mean().sort_values() > 0.0

Insects                       False
Crop                          False
Soil                          False
Category_of_Toxicant          False
Does_count                    False
Number_Weeks_does_not used    False
Crop_status                   False
Number_of_Weeks_Used           True
dtype: bool

Give all the column names whose NA value is greater than 10%

In [56]:
data.loc[:, data.isna().mean().sort_values() > 0.2].columns

Index([], dtype='object')

Drop all the Data with greater than 10% NA values 

In [57]:
data = data.drop(data.loc[:, data.isna().mean().sort_values() > 0.2].columns, axis=1)

In [58]:
data.shape

(200148, 8)

In [59]:
data

,Insects,Crop,Soil,Category_of_Toxicant,Does_count,Number_of_Weeks_Used,Number_Weeks_does_not used,Crop_status
0,188,Feed,clay,1,0,0.0,0,0
1,731,Food,clay,1,0,0.0,0,0
2,1212,Feed,clay,1,0,NaN,0,0
3,3516,Feed,clay,1,0,0.0,0,0
4,150,Food,clay,3,30,26.0,0,0
...,...,...,...,...,...,...,...,...
200143,1385,Food,clay,2,20,NaN,5,1
200144,1678,Feed,clay,3,40,38.0,0,1
200145,1575,Food,silt,2,50,NaN,26,1
200146,1212,Feed,clay,3,15,40.0,0,1


Drop All the rows with NA columns - Uncomment below to Drop NA Rows 

In [60]:
mean_of_Number_of_Weeks_Used = data['Number_of_Weeks_Used'].mean()

In [61]:
# data = data.dropna(axis=0).reset_index(drop=True)

# data = data.dropna(axis=0).reset_index(drop=True)
# data.apply(lambda x: x.fillna(x.mean()),axis=0)

data['Number_of_Weeks_Used'].fillna((data['Number_of_Weeks_Used'].mean()), inplace=True)


In [62]:
data.shape

(200148, 8)

Sum of NA for rows and cols 

In [63]:
data.isna().sum().sum()

0

## Feature Selection
Deal with Categorical Data

In [64]:
data

,Insects,Crop,Soil,Category_of_Toxicant,Does_count,Number_of_Weeks_Used,Number_Weeks_does_not used,Crop_status
0,188,Feed,clay,1,0,0.00000,0,0
1,731,Food,clay,1,0,0.00000,0,0
2,1212,Feed,clay,1,0,33.27621,0,0
3,3516,Feed,clay,1,0,0.00000,0,0
4,150,Food,clay,3,30,26.00000,0,0
...,...,...,...,...,...,...,...,...
200143,1385,Food,clay,2,20,33.27621,5,1
200144,1678,Feed,clay,3,40,38.00000,0,1
200145,1575,Food,silt,2,50,33.27621,26,1
200146,1212,Feed,clay,3,15,40.00000,0,1


In [65]:
{column: list(data[column].unique()) for column in data.columns if data.dtypes[column] == 'object'}

{'Crop': ['Feed', 'Food'], 'Soil': ['clay', 'silt']}

In [66]:
data.columns

Index(['Insects', 'Crop', 'Soil', 'Category_of_Toxicant', 'Does_count',
       'Number_of_Weeks_Used', 'Number_Weeks_does_not used', 'Crop_status'],
      dtype='object')

In [67]:
for item in data.columns:
  print(item,data[item].unique())

Insects [ 188  731 1212 3516  150  151  168  209  232  231  256  257  283  312
  311  342  375  411  410  448  488  489  531  577  626  625  677  789
  790  851  850  915  916  984 1056 1132 1296 1297 1385 1478 1576 1575
 1678 1785 1786 1898 2015 2139 2267 2402 2401 2542 2541 2688 2687 2840
 2999 3164 3165 3337 3702 3896  169  732 2016 2138 3895 3515 4096 4097
 3336]
Crop ['Feed' 'Food']
Soil ['clay' 'silt']
Category_of_Toxicant [1 3 2]
Does_count [ 0 30 40 60 20 25 45 50 15 10  5 35 70 55 80 90 85 65 75 95]
Number_of_Weeks_Used [ 0.         33.27620951 26.         20.         15.         14.
 19.         24.          8.         23.         25.         28.
 29.         22.         21.         17.         30.         35.
 27.         16.         32.         31.         33.         39.
 38.         42.         40.         41.         34.         44.
 36.         46.         45.         37.         50.         47.
 43.         49.         53.         51.         54.         61.
 48.      

In [68]:
data

,Insects,Crop,Soil,Category_of_Toxicant,Does_count,Number_of_Weeks_Used,Number_Weeks_does_not used,Crop_status
0,188,Feed,clay,1,0,0.00000,0,0
1,731,Food,clay,1,0,0.00000,0,0
2,1212,Feed,clay,1,0,33.27621,0,0
3,3516,Feed,clay,1,0,0.00000,0,0
4,150,Food,clay,3,30,26.00000,0,0
...,...,...,...,...,...,...,...,...
200143,1385,Food,clay,2,20,33.27621,5,1
200144,1678,Feed,clay,3,40,38.00000,0,1
200145,1575,Food,silt,2,50,33.27621,26,1
200146,1212,Feed,clay,3,15,40.00000,0,1


Get categorical data and Convert them into numbers 


In [69]:
categorical_cols  = []

for item in data.columns:
  # print(item,data[item].unique())
  if len(data[item].unique()) < 10:
    categorical_cols.append(item)

categorical_cols

['Crop', 'Soil', 'Category_of_Toxicant', 'Crop_status']

Applying indexes to categorical Columns

In [70]:
for cols in categorical_cols:
  list_data = list(data[cols].unique())
  print(" column ",cols)
  print( " distinct Values ",list_data)
  data[cols] = data[cols].apply(lambda x:list_data.index(x))

 column  Crop
 distinct Values  ['Feed', 'Food']
 column  Soil
 distinct Values  ['clay', 'silt']
 column  Category_of_Toxicant
 distinct Values  [1, 3, 2]
 column  Crop_status
 distinct Values  [0, 2, 1]


In [71]:
data

,Insects,Crop,Soil,Category_of_Toxicant,Does_count,Number_of_Weeks_Used,Number_Weeks_does_not used,Crop_status
0,188,0,0,0,0,0.00000,0,0
1,731,1,0,0,0,0.00000,0,0
2,1212,0,0,0,0,33.27621,0,0
3,3516,0,0,0,0,0.00000,0,0
4,150,1,0,1,30,26.00000,0,0
...,...,...,...,...,...,...,...,...
200143,1385,1,0,2,20,33.27621,5,2
200144,1678,0,0,1,40,38.00000,0,2
200145,1575,1,1,2,50,33.27621,26,2
200146,1212,0,0,1,15,40.00000,0,2


converting each data into float

In [72]:
# for column in data.columns:
#     try:
#         data[column] = data[column].astype(np.float)
#     except:
#         pass

In [73]:
data

,Insects,Crop,Soil,Category_of_Toxicant,Does_count,Number_of_Weeks_Used,Number_Weeks_does_not used,Crop_status
0,188,0,0,0,0,0.00000,0,0
1,731,1,0,0,0,0.00000,0,0
2,1212,0,0,0,0,33.27621,0,0
3,3516,0,0,0,0,0.00000,0,0
4,150,1,0,1,30,26.00000,0,0
...,...,...,...,...,...,...,...,...
200143,1385,1,0,2,20,33.27621,5,2
200144,1678,0,0,1,40,38.00000,0,2
200145,1575,1,1,2,50,33.27621,26,2
200146,1212,0,0,1,15,40.00000,0,2


## Encoding

In [74]:
target = 'Crop_status'


binary_features = ['Crop', 'Soil']

#  column  Crop
#  distinct Values  ['Feed', 'Food']
#  column  Soil
#  distinct Values  ['clay', 'silt']

# value of the Column to be said as one ( 1) 
binary_positives = ['Feed', 'clay']

# ordering 
# ordinal_features = ['emp_length']

# no ordering 
nominal_features = ['Category_of_Toxicant']

In [75]:
# Encoding functions

def binary_encode(df, column, positive_value):
    df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)

def ordinal_encode(df, column, ordering):
    df[column] = df[column].apply(lambda x: ordering.index(x))

def onehot_encode(df, column):
    dummies = pd.get_dummies(df[column])
    df_new = pd.concat([df, dummies], axis=1)
    df_new = df_new.drop(column, axis=1)
    return df_new

In [76]:
# Perform encoding

for feature, positive_value in zip(binary_features, binary_positives):
    binary_encode(data, feature, positive_value)

# not required here 
# ordinal_encode(data, 'emp_length', emp_ordering)

# adding a new columns 
for feature in nominal_features:
    data = onehot_encode(data, feature)

In [77]:
data.columns

Index([                   'Insects',                       'Crop',
                             'Soil',                 'Does_count',
             'Number_of_Weeks_Used', 'Number_Weeks_does_not used',
                      'Crop_status',                            0,
                                  1,                            2],
      dtype='object')

In [78]:
data

,Insects,Crop,Soil,Does_count,Number_of_Weeks_Used,Number_Weeks_does_not used,Crop_status,0,1,2
0,188,0,0,0,0.00000,0,0,1,0,0
1,731,0,0,0,0.00000,0,0,1,0,0
2,1212,0,0,0,33.27621,0,0,1,0,0
3,3516,0,0,0,0.00000,0,0,1,0,0
4,150,0,0,30,26.00000,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
200143,1385,0,0,20,33.27621,5,2,0,0,1
200144,1678,0,0,40,38.00000,0,2,0,1,0
200145,1575,0,0,50,33.27621,26,2,0,0,1
200146,1212,0,0,15,40.00000,0,2,0,1,0


In [79]:
data[target].value_counts()

2    66716
1    66716
0    66716
Name: Crop_status, dtype: int64

In [80]:
data.columns


Index([                   'Insects',                       'Crop',
                             'Soil',                 'Does_count',
             'Number_of_Weeks_Used', 'Number_Weeks_does_not used',
                      'Crop_status',                            0,
                                  1,                            2],
      dtype='object')

In [81]:
data

,Insects,Crop,Soil,Does_count,Number_of_Weeks_Used,Number_Weeks_does_not used,Crop_status,0,1,2
0,188,0,0,0,0.00000,0,0,1,0,0
1,731,0,0,0,0.00000,0,0,1,0,0
2,1212,0,0,0,33.27621,0,0,1,0,0
3,3516,0,0,0,0.00000,0,0,1,0,0
4,150,0,0,30,26.00000,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
200143,1385,0,0,20,33.27621,5,2,0,0,1
200144,1678,0,0,40,38.00000,0,2,0,1,0
200145,1575,0,0,50,33.27621,26,2,0,0,1
200146,1212,0,0,15,40.00000,0,2,0,1,0


Splitting and Scaling

In [82]:
y = data[target]
X = data.drop(target,axis=1)

In [83]:
y

0         0
1         0
2         0
3         0
4         0
         ..
200143    2
200144    2
200145    2
200146    2
200147    2
Name: Crop_status, Length: 200148, dtype: int64

In [84]:
y.shape

(200148,)

In [85]:
# Encoding label column

label_encoder = LabelEncoder()

y = label_encoder.fit_transform(y)

# y = y.apply(label_encoder.fit_transform) 

target_mappings = {index: label for index, label in enumerate(label_encoder.classes_)}
target_mappings


#  print le.inverse_transform(predictions_train)[:100]

{0: 0, 1: 1, 2: 2}

In [86]:
# def onehot_encode(df, column):
#     dummies = pd.get_dummies(df[column])
#     df_new = pd.concat([df, dummies], axis=1)
#     df_new = df_new.drop(column, axis=1)
#     return df_new

y = pd.get_dummies(y)  

In [87]:
y.shape

(200148, 3)

In [88]:
count = {}
for it in [0,1,2]:
  ic = 0
  for item in list(y[it]):
    if item == 1:
      ic = ic + 1
  count[it]=ic

print(count)

{0: 66716, 1: 66716, 2: 66716}


In [89]:
X

,Insects,Crop,Soil,Does_count,Number_of_Weeks_Used,Number_Weeks_does_not used,0,1,2
0,188,0,0,0,0.00000,0,1,0,0
1,731,0,0,0,0.00000,0,1,0,0
2,1212,0,0,0,33.27621,0,1,0,0
3,3516,0,0,0,0.00000,0,1,0,0
4,150,0,0,30,26.00000,0,0,1,0
...,...,...,...,...,...,...,...,...,...
200143,1385,0,0,20,33.27621,5,0,0,1
200144,1678,0,0,40,38.00000,0,0,1,0
200145,1575,0,0,50,33.27621,26,0,0,1
200146,1212,0,0,15,40.00000,0,0,1,0


In [90]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [91]:
X

array([[-1.61803485,  0.        ,  0.        , ...,  4.92690606,
        -0.89105971, -1.03636434],
       [-1.00745045,  0.        ,  0.        , ...,  4.92690606,
        -0.89105971, -1.03636434],
       [-0.46658286,  0.        ,  0.        , ...,  4.92690606,
        -0.89105971, -1.03636434],
       ...,
       [-0.05840213,  0.        ,  0.        , ..., -0.20296713,
        -0.89105971,  0.96491163],
       [-0.46658286,  0.        ,  0.        , ..., -0.20296713,
         1.12225924, -1.03636434],
       [ 0.57467157,  0.        ,  0.        , ..., -0.20296713,
         1.12225924, -1.03636434]])

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.85, random_state=42)

Modelling and Training 

In [93]:
y_test.shape

(30023, 3)

In [94]:
y.shape

(200148, 3)

In [95]:
X.shape

(200148, 9)

In [96]:
X_test.shape

(30023, 9)

In [97]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

In [98]:
# define the keras model
model = Sequential()
model.add(Dense(32, input_dim=9, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))

# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataseti
model.fit(X_train, y_train, epochs=20, batch_size=10, verbose=0)
# summarize the first 5 cases

In [99]:
# make class predictions with the model
predictions = model.predict(X_test)

predictions

array([[0.4031284 , 0.12834941, 0.46852216],
       [0.66216075, 0.02530829, 0.31253096],
       [0.36689225, 0.10468891, 0.52841884],
       ...,
       [0.04438066, 0.5413428 , 0.41427648],
       [0.269794  , 0.27418804, 0.45601794],
       [0.41399682, 0.35370672, 0.23229645]], dtype=float32)

In [100]:
# print(len(list(y_test[0])))
# print(len(list(y_test[1])))
# print(len(list(y_test[2])))

icounter = 0
index_list = {}

for item_index in y_test.values.tolist():
  item_index = item_index.index(max(item_index))
  if item_index not in index_list.keys():
    index_list[item_index] = 0
  index_list[item_index] += 1
print(index_list)

{2: 10126, 0: 9957, 1: 9940}


In [101]:
icounter = 0
index_list = {}

for item in predictions:
  item = list(item)
  item_index = item.index(max(item))
  # print(item)
  # print(item_index)
  # if item_index == 2:
  #   icounter = icounter + 1
  if item_index not in index_list.keys():
    index_list[item_index] = 0
  index_list[item_index] += 1
print(index_list)

{2: 5963, 0: 10763, 1: 13297}


In [102]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                320       
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 51        
Total params: 1,955
Trainable params: 1,955
Non-trainable params: 0
_________________________________________________________________


In [103]:
predictions.shape

(30023, 3)

In [104]:
# for i in range(5):
#   print(X_test[i], predictions[i], y_test[i])

In [105]:
# # As per the number of columns in X 
# inputs = tf.keras.Input(shape=(9,))
# x = tf.keras.layers.Dense(64, activation='relu')(inputs)
# x = tf.keras.layers.Dense(64, activation='relu')(x)
# # output classes are 3 
# outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

# model = tf.keras.Model(inputs=inputs, outputs=outputs)

# model.compile(
#     optimizer='adam',
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy']
# )

# batch_size = 1
# epochs = 30

# history = model.fit(
#     X_train,
#     y_train,
#     validation_split=0.2,
#     batch_size=batch_size,
#     epochs=epochs,
#     callbacks=[tf.keras.callbacks.ReduceLROnPlateau()]
# )

Results

In [106]:
# plt.figure(figsize=(14, 10))

# epochs_range = range(epochs)
# train_loss = history.history['loss']
# val_loss = history.history['val_loss']

# plt.plot(epochs_range, train_loss, label="Training Loss")
# plt.plot(epochs_range, val_loss, label="Validation Loss")

# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.title("Loss Over Time")
# plt.legend()

# plt.show()

In [107]:
# model.evaluate(X_test, y_test)

In [108]:
# predict
# predictions_of_test = model.predict(X_test)

In [109]:
# print(predictions_of_test.shape)
# print(predictions_of_test)
# print(y_test.shape)
# print(y_test)

In [110]:
# type(predictions_of_test)

In [111]:
# icounter = 0
# index_list = []
# for item in predictions_of_test:
#   icounter = icounter + 1 
#   item = list(item)
#   item_index = item.index(max(item))
#   print(item)
#   print(item_index)
#   if icounter > 10:
    
#     break
#   index_list.append(item_index)
# print(index_list)

In [112]:
# from sklearn import metrics
# Print the confusion matrix
# print(metrics.confusion_matrix(y_test.ravel(), predictions.ravel()))

In [113]:
X_test.shape

(30023, 9)

## Loading Test data

In [114]:
import pandas as pd 
url = 'https://raw.githubusercontent.com/connectnitish/ML_hack_Qipl/main/Sample_Submission_crop_without_season_test_data.csv'
# url = 'Sample_Submission_crop_without_season_test_data.csv'

data_test = pd.read_csv(url)
print(data_test)

          ID  Insects  ... Number_Weeks_does_not used Season
0          1      188  ...                          0      2
1          2      410  ...                          0      2
2          3      626  ...                          0      2
3          4      731  ...                          0      2
4          5      789  ...                          0      1
...      ...      ...  ...                        ...    ...
34995  34996      677  ...                          3      1
34996  34997      731  ...                         20      3
34997  34998      732  ...                         20      2
34998  34999      731  ...                          2      2
34999  35000      731  ...                         16      1

[35000 rows x 9 columns]


In [115]:
data_test = data_test.drop(['ID','Season'],axis=1)

In [116]:
data_test.shape

(35000, 7)

In [117]:
# predicted_values = model.predict(data_test)
# print(predicted_values)

## Encoding on Test data

In [118]:
binary_features = ['Crop', 'Soil']

#  column  Crop
#  distinct Values  ['Feed', 'Food']
#  column  Soil
#  distinct Values  ['clay', 'silt']

# value of the Column to be said as one ( 1) 
binary_positives = ['Feed', 'clay']

# ordering 
# ordinal_features = ['emp_length']

# no ordering 
nominal_features = ['Category_of_Toxicant']

In [119]:
# Encoding functions

def binary_encode(df, column, positive_value):
    df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)

def ordinal_encode(df, column, ordering):
    df[column] = df[column].apply(lambda x: ordering.index(x))

def onehot_encode(df, column):
    dummies = pd.get_dummies(df[column])
    df_new = pd.concat([df, dummies], axis=1)
    df_new = df_new.drop(column, axis=1)
    return df_new

In [120]:
# Perform encoding

for feature, positive_value in zip(binary_features, binary_positives):
    binary_encode(data_test, feature, positive_value)

# not required here 
# ordinal_encode(data, 'emp_length', emp_ordering)

# adding a new columns 
for feature in nominal_features:
    data_test = onehot_encode(data_test, feature)

In [121]:
data_test

,Insects,Crop,Soil,Does_count,Number_of_Weeks_Used,Number_Weeks_does_not used,1,2,3
0,188,1,0,0,NaN,0,1,0,0
1,410,1,0,0,0.0,0,1,0,0
2,626,1,1,0,0.0,0,1,0,0
3,731,1,1,0,0.0,0,1,0,0
4,789,0,1,0,0.0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
34995,677,0,0,40,27.0,3,0,1,0
34996,731,0,0,10,8.0,20,0,1,0
34997,732,0,0,10,10.0,20,0,1,0
34998,731,0,0,10,31.0,2,0,1,0


In [122]:
# for column in data_test.columns:
#     try:
#         data_test[column] = data_test[column].astype(np.float)
#     except:
#         pass

In [123]:
data_test.isna().mean().sort_values() > 0.0

Insects                       False
Crop                          False
Soil                          False
Does_count                    False
Number_Weeks_does_not used    False
1                             False
2                             False
3                             False
Number_of_Weeks_Used           True
dtype: bool

In [124]:
data_test['Number_of_Weeks_Used'].fillna((mean_of_Number_of_Weeks_Used), inplace=True)

In [125]:
data_test

,Insects,Crop,Soil,Does_count,Number_of_Weeks_Used,Number_Weeks_does_not used,1,2,3
0,188,1,0,0,33.27621,0,1,0,0
1,410,1,0,0,0.00000,0,1,0,0
2,626,1,1,0,0.00000,0,1,0,0
3,731,1,1,0,0.00000,0,1,0,0
4,789,0,1,0,0.00000,0,1,0,0
...,...,...,...,...,...,...,...,...,...
34995,677,0,0,40,27.00000,3,0,1,0
34996,731,0,0,10,8.00000,20,0,1,0
34997,732,0,0,10,10.00000,20,0,1,0
34998,731,0,0,10,31.00000,2,0,1,0


In [131]:
# data_test = pd.DataFrame(data_test).fillna(0)

# scaler = StandardScaler()
data_test = scaler.transform(data_test)

In [132]:
data_test

array([[-1.64273375,  2.88376832, -3.29707537, ..., 97.66370549,
        -5.02916369, -2.00813363],
       [-1.64242609,  2.88376832, -3.29707537, ..., 97.66370549,
        -5.02916369, -2.00813363],
       [-1.64212675,  2.88376832,  0.73660074, ..., 97.66370549,
        -5.02916369, -2.00813363],
       ...,
       [-1.64197985, -2.02834851, -3.29707537, ..., -1.11199192,
        -0.16220071, -2.00813363],
       [-1.64198124, -2.02834851, -3.29707537, ..., -1.11199192,
        -0.16220071, -2.00813363],
       [-1.64198124, -2.02834851, -3.29707537, ..., -1.11199192,
        -0.16220071, -2.00813363]])

In [128]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [129]:
# from tensorflow import keras
# # model = keras.models.load_model('path/to/location')

# model = None
# path_for_saving_models = '/content/gdrive/My Drive/QC_HACK/intermediate/models/'
# model = keras.models.load_model(path_for_saving_models + 'NN_Model_Valid_Acc_84.zip')
# predicted_values = model.predict(data_test)

In [133]:
predicted_values_unseen = model.predict(data_test)

In [134]:
print(predicted_values_unseen.shape)

(35000, 3)


In [ ]:
print(predicted_values_unseen)

{2: 5497, 1: 525, 0: 28978}


In [139]:
u_data = {}
Crop_status_list = []
ROW_ID_List = []
icounter=0

for item in predicted_values_unseen: 
  icounter = icounter + 1
  ROW_ID_List.append(icounter)
  temp_item = list(item)
  maxpos = temp_item.index(max(temp_item))
  Crop_status_list.append(maxpos)
  # if maxpos == 2:
  #   print("aa")
  if maxpos not in u_data.keys():
    u_data[maxpos] = maxpos

# ROW_ID	Crop_status
import pandas
df = pandas.DataFrame(data={"ROW_ID": ROW_ID_List, "Crop_status": Crop_status_list})
df.to_csv("./D_Neural_Network_Output.csv", sep=',',index=False)

print(u_data)

{2: 2, 1: 1, 0: 0}


In [ ]:
print(predicted_values)

## Save Model

In [ ]:
# Display the model's architecture
model.summary()

In [ ]:
cd /content/gdrive/My Drive/QC_HACK/intermediate

In [ ]:
# mkdir models

In [ ]:
# path_for_saving_models = '/content/gdrive/My Drive/QC_HACK/intermediate/models/'
# model.save(path_for_saving_models + 'NN_Model_Valid_Acc_84.zip')